In [4]:
from peak_character import find_nearest
#from peakaboo.peak_character import peakchar
from peak_character import convert_to_nm
from peak_character import peak_matrix
import numpy as np
import pandas as pd

In [5]:
def find_nearest(array,value):
    idx = (np.abs(array-value)).argmin()
    return idx

In [6]:
def test_find_nearest():
    array = 'string'
    value = 1
    try:
        find_nearest(array, value)
    except TypeError:
        pass
    else:
        print('TypeError not handled')

In [55]:
test_find_nearest()

In [72]:
def peakchar(data_nm, data_z_array, peak_index):
    """
    find the peak height and width
    
    Args:
        data_nm: wavelength array, numpy array
        data_z_array: data array
        peak_index: index of the peaks identified
    
    Returns:
        height: peak intensity, numpy array
        fwhm: widths pf peaks defined as full-width half-max, numpy array
        
    """
    num_peaks = len(peak_index)
    
    #array of peak height
    height = [data_z_array[idx] for idx in peak_index]
    
    #array of peak width
    half_height = [ht / 2 for ht in height]

    fwhm_idx_1 = np.empty_like(half_height)
    fwhm_idx_2 = np.empty_like(fwhm_idx_1)
    fwhm_nm_1 = np.empty_like(fwhm_idx_1)
    fwhm_nm_2 = np.empty_like(fwhm_idx_1)
    
    for i in range(num_peaks):
        #find the index and nm corresponding to half of the peak intensity
        #on the left side of the peak
        if i == 0:
            fwhm_idx_1[i] = find_nearest(data_z_array[0:peak_index[i]], half_height[i])
        else:
            fwhm_idx_1[i] = find_nearest(data_z_array[peak_index[i-1]:peak_index[i]], half_height[i]) + peak_index[i-1]

        fwhm_nm_1[i] = data_nm[int(fwhm_idx_1[i])]
        
        #find the index and nm corresponding to half of the peak intensity
        #on the right side of the peak   
        fwhm_idx_2[i] = find_nearest(data_z_array[peak_index[i]:], half_height[i]) + peak_index[i]
        fwhm_nm_2[i] = data_nm[int(fwhm_idx_2[i])]
    
    #calculate fwhm as the difference between the index/nm's on the left and
    #right side of the peak
    fwhm = fwhm_nm_2 - fwhm_nm_1

    return height, fwhm

In [73]:
import numpy as np
def test_peakchar():
    data_nm = 'string'
    data_z = np.zeros((5, 5))
    peak_idx = pd.DataFrame([[1, 2, 3], [1, 2, 3], \
                             [1, 2, 3], [1, 2, 3], \
                             [1, 2,3]])
    try:
        peakchar(data_nm, data_z, peak_idx)
    except TypeError:
        pass
    else:
        print('TypeError not handled', 'Check peak_finding output')
        
    data_nm = np.random.rand(144)
    data_z = np.random.rand(144)
    peak_idx = [5, 50]
    height, fwhm = peakchar(data_nm, data_z, peak_idx)

    if isinstance(height, list):
        pass
    else:
        raise Exception('Bad type', 'Height is not np array')
        
    if isinstance(fwhm, np.ndarray):
        pass
    else:
        raise Exception('Bad type', 'Fwhm is not np array')

In [74]:
test_peakchar()

In [84]:
def indexes(y, thres=0.3, min_dist=1):
    """
    Peak detection based on a gradient-method,
    adapted from peakutils.indexes


    Args:
    y : 1D data array, numpy array
    thres : lowest intensity to call a feature a peak,
    float between 0. and 1.
    min_dist : minimum distance between two peaks, int

    Returns:
    array of peak indices, numpy array

    """
    y_raw = y
    y = [abs(k) for k in y_raw]

    if isinstance(y, np.ndarray) and np.issubdtype(
            y.dtype, np.unsignedinteger):
        raise ValueError("y must be signed")
    
    assert type(thres) == 'float' or 'int', 'TypeError'
    assert type(min_dist) == 'float' or 'int', 'TypeError'
    
    thres = thres * (np.max(y) - np.min(y)) + np.min(y)
    min_dist = int(min_dist)
    
    
    # compute first order difference
    dy = np.diff(y)

    # propagate left and right values successively to fill all plateau pixels
    # (0-value)
    zeros, = np.where(dy == 0)

    # check if the singal is totally flat
    if len(zeros) == len(y) - 1:
        return np.array([])

    while len(zeros):
        # add pixels 2 by 2 to propagate left and right value onto the
        # zero-value pixel
        zerosr = np.hstack([dy[1:], 0.])
        zerosl = np.hstack([0., dy[:-1]])

        # replace 0 with right value if non zero
        dy[zeros] = zerosr[zeros]
        zeros, = np.where(dy == 0)

        # replace 0 with left value if non zero
        dy[zeros] = zerosl[zeros]
        zeros, = np.where(dy == 0)

    # find the peaks by using the first order difference
    peaks = np.where((np.hstack([dy, 0.]) < 0.)
                     & (np.hstack([0., dy]) > 0.)
                     & (y > thres))[0]

    # handle multiple peaks, respecting the minimum distance
    if peaks.size > 1 and min_dist > 1:
        highest = peaks[np.argsort(y[peaks])][::-1]
        rem = np.ones(y.size, dtype=bool)
        rem[peaks] = False

        for peak in highest:
            if not rem[peak]:
                sl = slice(max(0, peak - min_dist), peak + min_dist + 1)
                rem[sl] = True
                rem[peak] = False

        peaks = np.arange(y.size)[~rem]

    return peaks

In [76]:
def peak_matrix(nm_array, data_matrix, threshold, mindist):
    """
    find peaks in a data matrix
    and calculate the height and width of the peaks

    Args:
        nm_array: wavelength array
        data_matrix: two-way matrix
        threshold: threshold of normalized peak intensity to be identified
            as a peak, float between 0. and 1.
        mindist: minimum distance between two peaks, int

    Returns:
        three matrice that contains arrays of peak indices,
        peak heights and peak fwhm of each time-slice

    """

    peak_idx_matx = []
    peak_height_matx = []
    peak_fwhm_matx = []

    num_timeslice = np.shape(data_matrix)[1]

    for i in range(num_timeslice):
        data_timeslice = data_matrix.values[:, i]

        peak_idx = indexes(
            data_timeslice, threshold, mindist).tolist()
        peak_idx_matx.append(peak_idx)

        peak_height, peak_fwhm = peakchar(nm_array, data_timeslice, peak_idx)

        peak_height_matx.append(peak_height)
        peak_fwhm_matx.append(peak_fwhm)

        # convert index to nm
        peak_idx_nm = peak_idx_matx
        peak_fwhm_nm = peak_fwhm_matx

        # transfer to dataframe
        peak_idx_df = pd.DataFrame(peak_idx_nm)
        peak_height_df = pd.DataFrame(peak_height_matx)
        peak_fwhm_df = pd.DataFrame(peak_fwhm_nm)

    return peak_idx_df, peak_height_df, peak_fwhm_df

In [90]:
import numpy as np
def test_peak_matrix():
    import pandas as pd
    data_nm = np.random.rand(144)
    data_z = pd.DataFrame(np.random.rand(144, 700))
    threshold = 'z'
    mindist = 10
    try:
        peak_matrix(data_nm, data_z, threshold, mindist)
    except TypeError:
        pass
    else:
        print('TypeError not handled', 'Check threshold or mindist type')
    
    if isinstance(data_z, pd.core.frame.DataFrame):
        pass
    else:
        raise Exception('TypeError', 'Check smoothing function output type')
    return

In [91]:
test_peak_matrix()

In [88]:
data_z = pd.DataFrame(np.random.rand(144, 700))

In [89]:
type(data_z)

pandas.core.frame.DataFrame